In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
fileNameQrd = 'qrd_canonical_model.csv'
fileNameMatchRuleBook = 'ruleDict.json'
fileNameDocumentTypeNames = 'documentTypeNames.json'

fsMountName = '/mounted'

In [3]:
import pprint
import pandas as pd
import uuid
import json
import os
import glob
import re
import sys
from bs4 import NavigableString, BeautifulSoup
from collections import defaultdict
import random
import string

from utils.config import config
from utils.logger.logger import loggerCreator

# ePI Modules
from parse.rulebook.rulebook import StyleRulesDictionary

from parse.extractor.parser import parserExtractor
from match.matchDocument.matchDocument import MatchDocument
from documentAnnotation.documentAnnotation import DocumentAnnotation
from htmlDocTypePartitioner.partition import DocTypePartitioner
from extractContentBetweenHeadings.dataBetweenHeadingsExtractor import DataBetweenHeadingsExtractor
from fhirXmlGenerator.fhirXmlGenerator import FhirXmlGenerator
from fhirService.fhirService import FhirService
from utils.logger.matchLogger import MatchLogger
from languageInfo.documentTypeNames.documentTypeNames import DocumentTypeNames


class FolderNotFoundError(Exception):
    pass


def getRandomString(N):
    str_ = ''.join(random.choice(string.ascii_uppercase + string.digits
                                 + string.ascii_lowercase) for _ in range(N))
    return str_


def convertHtmlToJson(domain, procedureType, languageCode, fileNameHtml, fileNameQrd, fileNameLog, fsMountName, localEnv):

    if localEnv is True:
        pathSep = "\\"
        module_path = os.path.join(os.path.abspath(os.path.join('..')), 'data', 'converted_to_html', f'{domain}', f'{procedureType}', f'{languageCode}')
    else:
        pathSep = "/"
        module_path = os.path.join(f'{fsMountName}', 'data', 'converted_to_html', f'{domain}', f'{procedureType}', f'{languageCode}')
    
    # Generate output folder path
    output_json_path = module_path.replace('converted_to_html', 'outputJSON')

    """
        Check if input folder exists, else throw exception
    """
    if(os.path.exists(module_path)):
        filenames = glob.glob(os.path.join(module_path, fileNameHtml))

        # Create language specific folder in outputJSON folder if it doesn't exist
        if(not os.path.exists(output_json_path)):
            os.mkdir(output_json_path)
        logger = MatchLogger(f'Parser_{getRandomString(1)}', fileNameHtml,
                             domain, procedureType, languageCode, "HTML", fileNameLog)

        styleLogger = MatchLogger(
            f'Style Dictionary_{getRandomString(1)}', fileNameHtml, domain, procedureType, languageCode, "HTML", fileNameLog)

        styleRulesObj = StyleRulesDictionary(styleLogger,
                                             language=languageCode,
                                             fileName=fileNameQrd,
                                             domain=domain,
                                             procedureType=procedureType,
                                             fsMountName=fsMountName,
                                             localEnv=localEnv)

        parserObj = parserExtractor(config, logger, styleRulesObj.styleRuleDict,
                                    styleRulesObj.styleFeatureKeyList,
                                    styleRulesObj.qrd_section_headings)

        for input_filename in filenames:
          # if(input_filename.find('Kalydeco II-86-PI-clean')!=-1):
            output_filename = input_filename.replace('converted_to_html', 'outputJSON')
            style_filepath =  output_filename.replace('.html','.txt')
            style_filepath =  style_filepath.replace('.txtl','.txt')
            style_filepath =  style_filepath.replace('.htm','.txt')
            print("-------------",style_filepath,"-----------------")

            output_filename = output_filename.replace('.html', '.json')
            output_filename = output_filename.replace('.htm', '.json')
            print(input_filename, output_filename)
            parserObj.createPIJsonFromHTML(input_filepath=input_filename,
                                           output_filepath=output_filename,
                                           style_filepath = style_filepath,
                                           img_base64_dict=parserObj.convertImgToBase64(input_filename)
                                           )
            
        return output_filename.split(pathSep)[-1], style_filepath
    else:
        try:    
            raise FolderNotFoundError(module_path + " not found")
        except:  
            logger.logFlowCheckpoint("Folder For Language Code Not Found In Input File")
            logger.logException("Folder For Language Code Not Found In Input File")
        raise FolderNotFoundError(module_path + " not found")
        return None


def splitJson(domain, procedureType, languageCode, fileNameJson, fileNameQrd, fileNameLog, fsMountName, localEnv):

    styleLogger = MatchLogger(
        f'Style Dictionary_{getRandomString(1)}', fileNameJson, domain, procedureType, languageCode, "Json", fileNameLog)

    styleRulesObj = StyleRulesDictionary(styleLogger,
                                             language=languageCode,
                                             fileName=fileNameQrd,
                                             domain=domain,
                                             procedureType=procedureType,
                                             fsMountName=fsMountName,
                                             localEnv=localEnv)
    if localEnv is True:
        path_json = os.path.join(os.path.abspath(os.path.join(
                '..')), 'data', 'outputJSON', domain, procedureType, languageCode, fileNameJson)
    else:
        path_json = os.path.join(f'{fsMountName}', 'data', 'outputJSON', domain,  procedureType, languageCode, fileNameJson)
        
    partitionLogger = MatchLogger(
        f'Partition_{getRandomString(1)}', fileNameJson, domain, procedureType, languageCode, "Json", fileNameLog)

    partitioner = DocTypePartitioner(partitionLogger, localEnv)

    partitionedJsonPaths = partitioner.partitionHtmls(
        styleRulesObj.qrd_section_headings, path_json)

    return partitionedJsonPaths


def extractAndValidateHeadings(domain,
                               procedureType,
                               languageCode,
                               documentNumber,
                               fileNameDoc,
                               fileNameQrd,
                               fileNameMatchRuleBook,
                               fileNameDocumentTypeNames,
                               fileNameLog,
                               stopWordFilterLen=6,
                               isPackageLeaflet=False,
                               medName=None,
                               fsMountName='/mounted',
                               localEnv=False):

    if documentNumber == 0:
        topHeadingsConsidered = 4
        bottomHeadingsConsidered = 6
    elif documentNumber == 1:
        topHeadingsConsidered = 3
        bottomHeadingsConsidered = 5
    elif documentNumber == 2:
        topHeadingsConsidered = 5
        bottomHeadingsConsidered = 15
    else:
        topHeadingsConsidered = 5
        bottomHeadingsConsidered = 10

    print(f"Starting Heading Extraction For File :- {fileNameDoc}")
    logger = MatchLogger(f"Heading Extraction {fileNameDoc}", fileNameDoc, domain, procedureType, languageCode, documentNumber, fileNameLog)
    logger.logFlowCheckpoint("Starting Heading Extraction")

    stopWordlanguage = DocumentTypeNames(
        fileNameDocumentTypeNames=fileNameDocumentTypeNames,
        languageCode=languageCode,
        domain=domain,
        procedureType=procedureType,
        documentNumber=documentNumber,
        fsMountName=fsMountName,
        localEnv=localEnv).extractStopWordLanguage()

    matchDocObj = MatchDocument(
        logger,
        domain,
        procedureType,
        languageCode,
        documentNumber,
        fileNameDoc,
        fileNameQrd,
        fileNameMatchRuleBook,
        fileNameDocumentTypeNames,
        topHeadingsConsidered,
        bottomHeadingsConsidered,
        stopWordFilterLen,
        stopWordlanguage,
        isPackageLeaflet,
        medName,
        fsMountName,
        localEnv)
    df, coll = matchDocObj.matchHtmlHeaddingsWithQrd()

    return df, coll


def parseDocument(htmlDocPath, fileNameQrd, fileNameMatchRuleBook, fileNameDocumentTypeNames, fsMountName = '/mounted', localEnv= False, medName = None):
    
    if localEnv is True:
        pathSep = "\\"
        fileNameLog = os.path.join(os.path.abspath(os.path.join('..')),'data','FinalLog.txt')
    else:
        pathSep = "/"
        fileNameLog = os.path.join(f'{fsMountName}','data','FinalLog.txt')
    pathComponents = htmlDocPath.split(pathSep)
    print(pathComponents)
    fileNameHtml = pathComponents[-1]
    languageCode =  pathComponents[-2]
    procedureType = pathComponents[-3]
    domain = pathComponents[-4]

    #procedureType = "CAP"
    print(fileNameHtml,languageCode,procedureType)
    
    
    
    flowLogger =  MatchLogger(f"Flow Logger HTML_{getRandomString(1)}", fileNameHtml, domain, procedureType, languageCode, "HTML", fileNameLog)

    flowLogger.logFlowCheckpoint("Starting HTML Conversion To Json")
    ###Convert Html to Json
    fileNameJson, stylesFilePath = convertHtmlToJson( domain, procedureType, languageCode, fileNameHtml, fileNameQrd, fileNameLog, fsMountName, localEnv)
    
    flowLogger.logFlowCheckpoint("Completed HTML Conversion To Json")

    flowLogger.logFlowCheckpoint("Starting Json Split")

    ###Split Uber Json to multiple Jsons for each category.
    partitionedJsonPaths = splitJson(domain, procedureType, languageCode, fileNameJson, fileNameQrd, fileNameLog, fsMountName, localEnv)
    
    partitionedJsonPaths = [ path.split(pathSep)[-1] for path in partitionedJsonPaths]
    flowLogger.logFlowCheckpoint(str(partitionedJsonPaths))
    
    flowLogger.logFlowCheckpoint("Completed Json Split")
    
    flowLogger.logFlowCheckpoint("Started Processing Partitioned Jsons")
    
    for index, fileNamePartitioned in enumerate(partitionedJsonPaths):
        
        flowLogger.logFlowCheckpoint(f"\n\n\n\n||||||||||||||||||||||||||||||||{str(index)} ||||| {str(fileNamePartitioned)}||||||||||||||||||||||||||||||||\n\n\n\n")
        
        if index == 3:
            stopWordFilterLen = 100
            isPackageLeaflet = True
        else:
            stopWordFilterLen = 6
            isPackageLeaflet = False
            
        df, coll = extractAndValidateHeadings(domain,
                                    procedureType,
                                    languageCode,
                                    index,
                                    fileNamePartitioned,
                                    fileNameQrd,
                                    fileNameMatchRuleBook,
                                    fileNameDocumentTypeNames,
                                    fileNameLog,
                                    stopWordFilterLen=stopWordFilterLen,
                                    isPackageLeaflet=isPackageLeaflet,
                                    medName=medName,
                                    fsMountName=fsMountName,
                                    localEnv=localEnv)
        
        
        print(f"Completed Heading Extraction For File")
        flowLogger.logFlowCheckpoint("Completed Heading Extraction For File")
        
        print(f"Starting Document Annotation For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Document Annotation For File")
        documentAnnotationObj = DocumentAnnotation(fileNamePartitioned,'c20835db4b1b4e108828a8537ff41506','https://spor-sit.azure-api.net/pms/api/v2/',df,coll)
        try:
            pms_oms_annotation_data = documentAnnotationObj.processRegulatedAuthorizationForDoc()
            print(pms_oms_annotation_data)
        except:
            pms_oms_annotation_data = None
            print("Error Found")
            
        print(f"Completed Document Annotation")        
        flowLogger.logFlowCheckpoint("Completed Document Annotation")
        
        print(f"Starting Extracting Content Between Heading For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Extracting Content Between Heading")
        
        extractContentlogger =  MatchLogger(f'ExtractContentBetween_{index}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        extractorObj = DataBetweenHeadingsExtractor(extractContentlogger, coll, domain, procedureType, languageCode, fsMountName, localEnv)
        dfExtractedHierRR = extractorObj.extractContentBetweenHeadings(fileNamePartitioned)
        
        print(f"Completed Extracting Content Between Heading")        
        flowLogger.logFlowCheckpoint("Completed Extracting Content Between Heading")
        
        xmlLogger =  MatchLogger(f'XmlGeneration_{index}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        fhirXmlGeneratorObj = FhirXmlGenerator(xmlLogger, pms_oms_annotation_data, stylesFilePath, medName, fsMountName, localEnv)
        fileNameXml = fileNamePartitioned.replace('.json','.xml')
        generatedXml = fhirXmlGeneratorObj.generateXml(dfExtractedHierRR, fileNameXml)
        
        fhirServiceLogger =  MatchLogger(f'XML Submission Logger_{index}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)

        fhirServiceObj = FhirService(fhirServiceLogger, generatedXml, fsMountName, localEnv)
        fhirServiceObj.submitFhirXml()
        print(f"Created XML File For :- {fileNamePartitioned}")        

        #return df,coll,dfExtractedHierRR
    
    flowLogger.logFlowCheckpoint("Completed Processing Partitioned Jsons")


C:\Users\vipsharm\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vipsharm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
parseDocument("F:\\Projects\\EMA\\Repository\\EMA EPI PoC\\function_code\\data\\converted_to_html\\H\\CAP\\en\\Adakveo-4874 EN PI.html", fileNameQrd, fileNameMatchRuleBook, fileNameDocumentTypeNames, medName = "Adakveo", fsMountName = '/mounted', localEnv= True)

2021-05-11 01:54:31,174 : Flow Logger HTML_p : Starting HTML Conversion To Json | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-11 01:54:31,182 : Style Dictionary_3 : Reading style dictionary in file: rule_dictionary_en.json | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-11 01:54:31,260 : Style Dictionary_3 : Qrd Section Keys Retrieved For Style Dictionary: ANNEX I, ANNEX II, ANNEX III, B. PACKAGE LEAFLET | H | CAP |  en | HTML | Adakveo-4874 EN PI.html


['F:', 'Projects', 'EMA', 'Repository', 'EMA EPI PoC', 'function_code', 'data', 'converted_to_html', 'H', 'CAP', 'en', 'Adakveo-4874 EN PI.html']
Adakveo-4874 EN PI.html en CAP
------------- F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\en\Adakveo-4874 EN PI.txt -----------------
F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\converted_to_html\H\CAP\en\Adakveo-4874 EN PI.html F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\en\Adakveo-4874 EN PI.json


2021-05-11 01:54:31,836 : Parser_H : Style Information Stored In File: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\en\Adakveo-4874 EN PI.txt | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-11 01:54:34,086 : Parser_H : Writing to file: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\en\Adakveo-4874 EN PI.json | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-11 01:54:34,193 : Flow Logger HTML_p : Completed HTML Conversion To Json | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-11 01:54:34,194 : Flow Logger HTML_p : Starting Json Split | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-11 01:54:34,199 : Style Dictionary_m : Reading style dictionary in file: rule_dictionary_en.json | H | CAP |  en | Json | Adakveo-4874 EN PI.json
2021-05-11 01:54:34,237 : Style Dictionary_m : Qrd Section Keys Retrieved For Style Dictionary: ANNEX I, ANNEX II, ANNEX III, B. PACKAGE LEAFLET | H | CAP |  en | Json | Adak

Starting Heading Extraction For File :- Adakveo-4874 EN PI_SmPC.json
File being processed: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\en\Adakveo-4874 EN PI_SmPC.json
--------------------------------------------
SmPC


2021-05-11 01:54:34,938 : Heading Extraction Adakveo-4874 EN PI_SmPC.json : Started Extracting Heading | H | CAP |  en | 0 | Adakveo-4874 EN PI_SmPC.json
2021-05-11 01:54:35,149 : Heading Extraction Adakveo-4874 EN PI_SmPC.json : Match Passed | H | CAP |  en | 0 | Adakveo-4874 EN PI_SmPC.json | Doc txt :- 'SUMMARY OF PRODUCT CHARACTERISTICS' | Qrd txt :- 'SUMMARY OF PRODUCT CHARACTERISTICS' | Matched :- 'True'
2021-05-11 01:54:35,160 : Heading Extraction Adakveo-4874 EN PI_SmPC.json : Validation Passed As This The First Heading | H | CAP |  en | 0 | Adakveo-4874 EN PI_SmPC.json | currHeadId :- '20001' | currParentHeadId :- 'nan' | prevParentHeadId :- ''
 adverse reactions.' | Qrd txt :- 'qthis medicinal product is subject to additional monitoring. this will allow quick identification of new safety information. healthcare professionals are asked to report any suspected adverse reactions. see section 4.8 for how to report adverse reactions.' | Matched :- 'True'


...

2021-05-11 01:55:59,999 : Heading Extraction Adakveo-4874 EN PI_ PACKAGE LEAFLET.json : Match Failed | H | CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json | Doc txt :- 'Preparing the infusion' | Qrd txt :- 'Pregnancy <and> <,> breast-feeding <and fertility>' | Matched :- 'False'
2021-05-11 01:56:00,097 : Heading Extraction Adakveo-4874 EN PI_ PACKAGE LEAFLET.json : Match Failed | H | CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json | Doc txt :- 'Preparing the infusion' | Qrd txt :- 'This leaflet was last revised in <{MM/YYYY}><{month YYYY}>.' | Matched :- 'False'
 aseptic techniques.' | Qrd txt :- 'Pregnancy <and> <,> breast-feeding <and fertility>' | Matched :- 'False'AP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json | Doc txt :- 'The diluted
 aseptic techniques.' | Qrd txt :- 'This leaflet was last revised in <{MM/YYYY}><{month YYYY}>.' | Matched :- 'False'| 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json | Doc txt :- 'The diluted
 of crizanlizumab is administered

 overall.' | Qrd txt :- 'Pregnancy <and> <,> breast-feeding <and fertility>' | Matched :- 'False'led | H | CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json | Doc txt :- 'Chemical and
 overall.' | Qrd txt :- 'This leaflet was last revised in <{MM/YYYY}><{month YYYY}>.' | Matched :- 'False' CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json | Doc txt :- 'Chemical and
2021-05-11 01:56:19,591 : Heading Extraction Adakveo-4874 EN PI_ PACKAGE LEAFLET.json : Match Failed | H | CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json | Doc txt :- 'Administration' | Qrd txt :- 'Pregnancy <and> <,> breast-feeding <and fertility>' | Matched :- 'False'
2021-05-11 01:56:19,656 : Heading Extraction Adakveo-4874 EN PI_ PACKAGE LEAFLET.json : Match Failed | H | CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json | Doc txt :- 'Administration' | Qrd txt :- 'This leaflet was last revised in <{MM/YYYY}><{month YYYY}>.' | Matched :- 'False'
 chloride 9 mg/ml (0.9%) solution for injection



Heading Not Found 
 ['q This medicine is subject to additional monitoring. This will allow quick identification of new safety information. You can help by reporting any side effects you may get. See the end of section 4 for how to report side effects.', 'Do not <take> <use> X', 'X with <food> <and> <,> <drink> <and> <alcohol>', 'How to <take> <use> X ', 'Use in children <and adolescents>', 'If you <take> <use> more X than you should', 'If you forget to <take> <use> X>', 'If you stop <taking> <using> X>', 'Additional side effects in children <and adolescents>', 'Marketing Authorisation Holder and Manufacturer']


dict_keys(['q This medicine is subject to additional monitoring. This will allow quick identification of new safety information. You can help by reporting any side effects you may get. See the end of section 4 for how to report side effects.', '1. What Adakveo is and what it is used for', '2. What you need to know before you <take> <use> Adakveo ', '4. Possible side effects',

2021-05-11 01:56:21,734 : XmlGeneration_3 : Writing to File:Adakveo-4874 EN PI_ PACKAGE LEAFLET.xml | H | CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json
2021-05-11 01:56:23,485 : XML Submission Logger_3 : Initiating Submission To FHIR Server | H | CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json
2021-05-11 01:56:23,490 : XML Submission Logger_3 : POST sucessful: XML added with id: 50733669-e9a4-499b-86e9-9518106bd968 | H | CAP |  en | 3 | Adakveo-4874 EN PI_ PACKAGE LEAFLET.json
2021-05-11 01:56:23,496 : Flow Logger HTML_p : Completed Processing Partitioned Jsons | H | CAP |  en | HTML | Adakveo-4874 EN PI.html


POST sucessful: XML added with id 50733669-e9a4-499b-86e9-9518106bd968
Created XML File For :- Adakveo-4874 EN PI_ PACKAGE LEAFLET.json


In [4]:
parseDocument("F:\\Projects\\EMA\\Repository\\EMA EPI PoC\\function_code\\data\\converted_to_html\\H\\CAP\\bg\\emea-combined-h-2494-bg.htm", fileNameQrd, fileNameMatchRuleBook, fileNameDocumentTypeNames, medName = "Kalydeco", fsMountName = '/mounted', localEnv= True)

2021-05-11 02:37:10,668 : Flow Logger HTML_T : Starting HTML Conversion To Json | H | CAP |  bg | HTML | emea-combined-h-2494-bg.htm
2021-05-11 02:37:10,687 : Style Dictionary_7 : Reading style dictionary in file: rule_dictionary_bg.json | H | CAP |  bg | HTML | emea-combined-h-2494-bg.htm
2021-05-11 02:37:10,790 : Style Dictionary_7 : Qrd Section Keys Retrieved For Style Dictionary: ПРИЛОЖЕНИЕ I, ПРИЛОЖЕНИЕ II, ПРИЛОЖЕНИЕ III, B.  ЛИСТОВКА | H | CAP |  bg | HTML | emea-combined-h-2494-bg.htm


['F:', 'Projects', 'EMA', 'Repository', 'EMA EPI PoC', 'function_code', 'data', 'converted_to_html', 'H', 'CAP', 'bg', 'emea-combined-h-2494-bg.htm']
emea-combined-h-2494-bg.htm bg CAP
------------- F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\bg\emea-combined-h-2494-bg.txt -----------------
F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\converted_to_html\H\CAP\bg\emea-combined-h-2494-bg.htm F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\bg\emea-combined-h-2494-bg.json


2021-05-11 02:37:15,282 : Parser_2 : Style Information Stored In File: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\bg\emea-combined-h-2494-bg.txt | H | CAP |  bg | HTML | emea-combined-h-2494-bg.htm
2021-05-11 02:37:32,258 : Parser_2 : Writing to file: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\bg\emea-combined-h-2494-bg.json | H | CAP |  bg | HTML | emea-combined-h-2494-bg.htm
2021-05-11 02:37:33,083 : Flow Logger HTML_T : Completed HTML Conversion To Json | H | CAP |  bg | HTML | emea-combined-h-2494-bg.htm
2021-05-11 02:37:33,086 : Flow Logger HTML_T : Starting Json Split | H | CAP |  bg | HTML | emea-combined-h-2494-bg.htm
2021-05-11 02:37:33,141 : Style Dictionary_X : Reading style dictionary in file: rule_dictionary_bg.json | H | CAP |  bg | Json | emea-combined-h-2494-bg.json
2021-05-11 02:37:33,254 : Style Dictionary_X : Qrd Section Keys Retrieved For Style Dictionary: ПРИЛОЖЕНИЕ I, ПРИЛОЖЕНИЕ II, ПРИЛОЖЕНИЕ III, B.

Starting Heading Extraction For File :- emea-combined-h-2494-bg_SmPC.json
File being processed: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\bg\emea-combined-h-2494-bg_SmPC.json
--------------------------------------------
SmPC


2021-05-11 02:37:35,419 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Started Extracting Heading | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json
 НА ПРОДУКТА' | Qrd txt :- 'КРАТКА ХАРАКТЕРИСТИКА НА ПРОДУКТА' | Matched :- 'True'atch Passed | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | Doc txt :- 'КРАТКА
2021-05-11 02:37:35,942 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Validation Passed As This The First Heading | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | currHeadId :- '1' | currParentHeadId :- 'nan' | prevParentHeadId :- ''
 ПРОДУКТ' | Qrd txt :- '1. ИМЕ НА ЛЕКАРСТВЕНИЯ ПРОДУКТ' | Matched :- 'True'son : Match Passed | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | Doc txt :- '1.       ИМЕ
2021-05-11 02:37:36,003 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Validation Flow Is Broken | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | currHeadId :- '3' | currParentHeadId :- '1.0' | prevParentHeadId :- '

...

----------------------------------
RemovedByStyle
----------------------------------


 безопасност' | Qrd txt :- 'Клинична ефикасност и безопасност' | Matched :- 'True'atch Passed | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | Doc txt :- 'Клинична
2021-05-11 02:38:18,707 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Validation Flow Is Broken | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | currHeadId :- '36' | currParentHeadId :- '33.0' | prevParentHeadId :- '1'
2021-05-11 02:38:18,718 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Validation Passed | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | currHeadId :- '36' | currParentHeadId :- '33.0' | prevParentHeadId :- '1'
2021-05-11 02:38:18,724 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Validation Failed By Style | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | currHeadId :- '36' | currParentHeadId :- '33.0' | prevParentHeadId :- '1'


----------------------------------
RemovedByStyle
----------------------------------


 каналопатии ' | Qrd txt :- '6.5 Вид и съдържание на опаковката <и специални приспособления за употреба, приложение или имплантиране>' | Matched :- 'False' txt :- 'Проучвания 1
 лечение. ' | Qrd txt :- '6.5 Вид и съдържание на опаковката <и специални приспособления за употреба, приложение или имплантиране>' | Matched :- 'False'Doc txt :- 'Първичната
 са дадени в Таблица 6.' | Qrd txt :- '6.5 Вид и съдържание на опаковката <и специални приспособления за употреба, приложение или имплантиране>' | Matched :- 'False'езултатите
 проучванията1 и2' | Qrd txt :- '6.5 Вид и съдържание на опаковката <и специални приспособления за употреба, приложение или имплантиране>' | Matched :- 'False' :- 'Таблица 6:
 събитията' | Qrd txt :- '6.5 Вид и съдържание на опаковката <и специални приспособления за употреба, приложение или имплантиране>' | Matched :- 'False'Doc txt :- 'ДИ:
 събитията' | Qrd txt :- '6.6 Специални предпазни мерки при изхвърляне <и работа>' | Matched :- 'False' |  bg | 0 | emea-combined

KeyboardInterrupt: 